In [26]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
import matplotlib.pyplot as plt

from scipy.stats import gaussian_kde
import h2o
from h2o.automl import H2OAutoML

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option("display.max_rows", 8)

In [5]:
df = pd.read_pickle("../../data/df.pkl")

In [4]:
df = df.loc[df.type_bool == True]
df

,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,...,f14,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC
ID,,,,,,,,,,,,,,,,,,,,,
SN10005,"[[10.027000000001863, 4.996, 8.081], [33.94500...","[[0.015000000006693881, 0.2977, 2.206], [18.94...","[[0.0, 1.033, 1.325], [13.015000000006694, 0.4...","[[0.03500000000349246, 1.826, 3.029], [48.875,...","[0.0, 1.0381919191919462, 2.0763838383838924, ...","([10.027000000001863, 10.782565656567513, 11.5...","([0.015000000006693881, 1.0530404040470573, 2....","([0.0, 1.0281717171717606, 2.056343434343521, ...","([0.03500000000349246, 1.0222929292964054, 2.0...","[2.000004411755352, 2.000003547336676, 2.00000...",...,-26.554909,15.768490,16.436792,17.114795,-20.292951,-24.222566,-11.295562,Ia,True,1.08327
SN10007,"[[0.0, 6.266, 10.48], [10.105999999999767, 17....","[[0.03200000000651926, 7.161, 9.313], [2.01600...","[[2.0, 1.24, 2.429], [10.121000000006461, 20.0...","[[0.9770000000062282, 0.6586, 5.329], [15.9930...","[0.0, 1.5136565656566285, 3.027313131313257, 4...","([0.0, 1.5132626262626456, 3.0265252525252913,...","([0.03200000000651926, 1.5451717171782005, 3.0...","([2.0, 3.49314141414145, 4.9862828282829, 6.47...","([0.9770000000062282, 2.480787878794107, 3.984...","[11.341091517194192, 8.812204685737349, 6.9798...",...,-99.877031,21.670852,24.155364,-26.622112,-27.616185,-19.981432,49.299250,Ia,True,0.82331
SN100361,"[[6.957000000002154, 16.02, 23.92], [7.9690000...","[[0.023000000001047738, 4.903, 3.324], [6.9840...","[[0.0, 0.7103, 2.626], [5.9760000000023865, 5....","[[8.055000000000291, 2.567, 2.665], [15.035000...","[0.0, 1.3031111111111275, 2.606222222222255, 3...","([6.957000000002154, 8.15910101010318, 9.36120...","([0.023000000001047738, 1.315494949496019, 2.6...","([0.0, 1.3026767676768058, 2.6053535353536117,...","([8.055000000000291, 9.27674747474778, 10.4984...","[8.233549869515556, 9.464488874691614, 10.4511...",...,102.074783,-95.727930,4.861555,-29.749073,23.442440,5.441237,12.503390,Ia,True,0.90012
SN100471,"[[0.0, 3.79, 4.764], [11.969000000004598, 1.67...","[[0.023999999997613486, 2.202, 3.146], [3.0429...","[[0.008000000001629815, 0.4567, 2.129], [3.004...","[[0.046999999998661224, 6.92, 2.486], [3.07500...","[0.0, 1.4638181818181601, 2.9276363636363203, ...","([0.0, 1.3621414141414037, 2.7242828282828073,...","([0.023999999997613486, 1.4875757575733735, 2....","([0.008000000001629815, 1.3701818181834458, 2....","([0.046999999998661224, 1.3283737373723963, 2....","[5.63245418874034, 5.603687474301877, 5.594769...",...,-40.831628,28.629443,12.801879,-0.555542,13.065768,-21.776767,9.358561,Ia,True,0.92640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN99404,"[[0.0, 6.586, 10.48], [15.980999999999767, 166...","[[22.0, 3.161, 11.98], [40.97700000000623, 15....","[[0.01600000000325963, 8.663, 7.843], [2.0, 1....","[[15.993000000002212, 5.828, 6.542], [29.95700...","[0.0, 1.745343434343478, 3.490686868686956, 5....","([0.0, 1.7352424242424678, 3.4704848484849355,...","([22.0, 23.382101010101024, 24.76420202020205,...","([0.01600000000325963, 1.7511212121244906, 3.4...","([15.993000000002212, 17.57679797980021, 19.16...","[475.9257034984372, 265.54157358525424, 120.22...",...,496.744492,-1269.560018,826.117566,-2224.164801,66.007045,341.497556,-173.911137,Ia,True,0.89529
SN99438,"[[0.0, 2.33, 2.55], [7.894000000000233, 3.805,...","[[0.030999999995401595, 0.006543, 3.683], [7.9...","[[0.014999999999417923, 0.4446, 1.3], [14.0039...","[[0.06199999999807915, 1.497, 4.486], [7.97199...","[0.0, 1.7255757575757058, 3.4511515151514116, ...","([0.0, 1.6948383838383538, 3.3896767676767077,...","([0.030999999995401595, 1.75597979797521, 3.48...","([0.014999999999417923, 1.7298888888882826, 3....","([0.06199999999807915, 1.7869494949475417, 3.5...","[6.789900493579756, 7.110462975693107, 7.57937...",...,-3.768615,-34.417710,32.878026,-33.942007,34.613437,61.324861,-51.210871,Ia,True,0.75966
SN99457,"[[0.0, 10.06, 4.

In [36]:
X = df.loc[:,'f1':'REDSHIFT_SPEC']
y = df.loc[:,'REDSHIFT_SPEC']

#### Getting h2o best result and making a prediction to list

In [ ]:
h2o.init(ip="localhost", port=54323)
best_model = h2o.load_model("/home/fmoliveira/Documents/Pessoal/Supernova_Identification_Article/models/h2o_ML_model_leader/StackedEnsemble_AllModels_AutoML_20201023_063859")
#Trained with X first half
preds = best_model.predict(h2o.H2OFrame(X.iloc[int(len(X)/2):-1].loc[:,'f1':'f20']))

In [61]:
df_sigma_mad = df.iloc[int((len(df)/2)+1):]
df_sigma_mad['zphot'] = preds.as_data_frame()['predict'].tolist()
df_sigma_mad

/home/fmoliveira/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,...,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC,zphot
ID,,,,,,,,,,,,,,,,,,,,,
SN509279,"[[18.89900000000489, 0.2945, 4.365], [26.90600...","[[0.008000000001629815, 7.418, 8.133], [32.824...","[[0.0, 3.166, 3.995], [5.867000000005646, 0.43...","[[18.93000000000029, 0.689, 5.952], [26.938000...","[0.0, 1.623303030303052, 3.246606060606104, 4....","([18.89900000000489, 20.331404040408902, 21.76...","([0.008000000001629815, 1.6212020202036719, 3....","([0.0, 1.613202020202042, 3.226404040404084, 4...","([18.93000000000029, 20.24135353535383, 21.552...","[20.927748638000686, 22.041179082094693, 22.99...",...,49.884274,82.560061,-94.391054,-86.511354,26.779625,-60.881068,Ia,True,0.65538,0.669640
SN509387,"[[10.089999999996508, 1.817, 25.57], [37.91399...","[[2.0, 2.957, 4.31], [10.944999999999709, 28.0...","[[0.0, 5.014, 7.842], [42.91000000000349, 24.5...","[[15.976999999998952, 0.967, 6.541], [29.94099...","[0.0, 1.594262626262648, 3.188525252525296, 4....","([10.089999999996508, 11.582141414137938, 13.0...","([2.0, 3.49313131313131, 4.98626262626262, 6.4...","([0.0, 1.594262626262648, 3.188525252525296, 4...","([15.976999999998952, 17.248141414140374, 18.5...","[6.575579676585663, 4.2190314913182885, 2.5365...",...,135.784689,196.657130,-74.459661,-83.874633,25.921586,-63.886573,Ia,True,0.45100,0.595281
SN50950,"[[0.0, 3.267, 4.768], [2.997000000003027, 6.71...","[[0.023999999997613486, 48.76, 3.284], [3.0429...","[[0.008000000001629815, 29.71, 2.211], [3.0040...","[[0.046999999998661224, 49.9, 2.656], [3.07500...","[0.0, 0.8286464646464652, 1.6572929292929304, ...","([0.0, 0.8285252525252285, 1.657050505050457, ...","([0.023999999997613486, 0.8504646464623306, 1....","([0.008000000001629815, 0.8365656565672706, 1....","([0.046999999998661224, 0.8735454545441292, 1....","[8.898194799968902, 9.262242225808274, 9.57102...",...,40.663644,76.159330,40.570551,12.182103,27.299124,10.842452,Ia,True,0.51480,0.526969
SN509630,"[[0.0, 12.41, 9.412], [1.9839999999967404, 1.6...","[[0.014999999999417923, 31.1, 6.866], [2.00699...","[[0.006999999997788109, 29.98, 7.202], [2.0, 2...","[[0.023000000001047738, 21.16, 7.932], [2.0149...","[0.0, 1.1505252525251979, 2.3010505050503958, ...","([0.0, 1.1505252525251979, 2.3010505050503958,...","([0.014999999999417923, 1.0752525252519078, 2....","([0.006999999997788109, 1.1269090909068709, 2....","([0.023000000001047738, 1.0833333333343622, 2....","[16.000170811346788, 16.460013780148202, 16.84...",...,-17.538851,32.414132,29.735733,16.959055,8.597079,-1.332496,Ia,True,0.68329,0.577279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN99404,"[[0.0, 6.586, 10.48], [15.980999999999767, 166...","[[22.0, 3.161, 11.98], [40.97700000000623, 15....","[[0.01600000000325963, 8.663, 7.843], [2.0, 1....","[[15.993000000002212, 5.828, 6.542], [29.95700...","[0.0, 1.745343434343478, 3.490686868686956, 5....","([0.0, 1.7352424242424678, 3.4704848484849355,...","([22.0, 23.382101010101024, 24.76420202020205,...","([0.01600000000325963, 1.7511212121244906, 3.4...","([15.993000000002212, 17.57679797980021, 19.16...","[475.9257034984372, 265.54157358525424, 120.22...",...,-1269.560018,826.117566,-2224.164801,66.007045,341.497556,-173.911137,Ia,True,0.89529,0.839917
SN99438,"[[0.0, 2.33, 2.55], [7.894000000000233, 3.805,...","[[0.030999999995401595, 0.006543, 3.683], [7.9...","[[0.014999999999417923, 0.4446, 1.3], [14.0039...","[[0.06199999999807915, 1.497, 4.486], [7.97199...","[0.0, 1.7255757575757058, 3.4511515151514116, ...","([0.0, 1.6948383838383538, 3.3896767676767077,...","([0.030999999995401595, 1.75597979797521, 3.48...","([0.014999999999417923, 1.7298888888882826, 3....","([0.06199999999807915, 1.7869494949475417, 3.5...","[6.789900493579756, 7.110462975693107, 7.57937...",...,-34.417710,32.878026,-33.942007,34.613437,61.324861,-51.210871,Ia,True,0.75966,0.485393
SN99457,"[[0.0, 10.06, 4.768], 

#### Deltas and Sigmas calculus

In [71]:
get_delta_z = lambda zphot, zspec : (zphot - zspec)/(1 + zspec)

In [87]:
delta_z = get_delta_z(df_sigma_mad.REDSHIFT_SPEC.values, df_sigma_mad.zphot.values)
delta_z

array([-0.00854058, -0.09044255, -0.0079693 , ...,  0.18464257,
        0.01135406, -0.26343139])

In [86]:
median_delta_z = np.median(delta_z)
median_delta_z

0.0007232058109612594

In [85]:
MAD = np.median(np.abs(delta_z - median_delta_z))
MAD

0.1213389099225449

In [84]:
sigma_MAD = 1.4826*MAD
sigma_MAD

0.17989706785116505

In [95]:
n_fraction = sum(np.abs(delta_z) > 0.05)/len(delta_z)
n_fraction

0.7730496453900709

In [98]:
n_fraction2 = sum(np.abs(delta_z) > 5*sigma_MAD)/len(delta_z)
n_fraction2

0.0

In [99]:
5*sigma_MAD

0.8994853392558253

In [100]:
103306/0.8

129132.5